## Advanced Machine Learning Workshop
### Assignment 5: CarNet
### Description of the Solution
### Priyanka Rotte
 
#### Individual Models:
For individual models, I’m using transfer learning from the EfficientNetV2B1 model. I tried various models available in the Keras library based on Size, Accuracy, Parameters, Time, etc. and EfficientNetV2B1 performed the best for the given data. I’m removing the fully-connected layer at the top of the model and using global average pooling on the output of the layer before. Average pooling worked better than flattening, as the output shape was getting too high after flattening. <br> 
After EfficientNetV2B1, I’m using two separate models for vehicles and signals, as to enable the models to learn features specific to the type. After several experiments, I’m going with two dense layers of sizes 64 and 32, with ReLU activations. Finally, there’s an output dense layer with 1 neuron. I’m not using any dropouts or regularization as the ensemble method with bootstrap replicates should prevent overfitting. I’m using Adam optimizer with a 0.001 learning rate, MSE loss, and early stopping.
#### Ensemble Model (Blender):
I’m training 7 individual models with the same architecture with different kernel initializations and bootstrap replicates i.e., random resampling of the dataset with replacement. <br>
After training the individual models, I’m training an ensemble model with the outputs of these 7 models as inputs to the ensemble. Instead of just getting an average, I’m using a blender i.e., a neural network on top of the individual models. Here too, I’m using two separate models for vehicles and signals, just with one dense layer of 128 and ReLU, and one dropout (even dropout will act as an ensemble of models and prevent overfitting). This architecture gave the lowest validation MSE. Finally, there’s an output dense layer with 1 neuron. The training is performed similarly as for the individual models. I’m saving the final model with the least validation loss. Finally, I’m getting a validation MSE of 4.29 and a test MSE of 4.36.
#### Submission:
- Jupyter notebook with code for the model
- Parquet file with predictions on score
- Zip file
    1. individual_models has individual folders for each model with their model definition, parameters and predictions on train, validate, test, and score.
    2. ensemble_model has ensemble model definition, parameters, and predictions on score set. Folder inside, ensemble_alternate has the other way of creating the same ensemble.
- PDF with solution description

In [1]:
import pandas as pd
import random
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
import json

%config InlineBackend.figure_format = 'retina'

import math
import seaborn as sns
sns.set_context('talk')

In [2]:
# This configures the GPU to be used by Tensorflow.
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

2022-10-31 04:05:09.954006: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 04:05:10.009021: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 04:05:10.010090: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


## Training Images and Metadata

In [3]:
train_metadata = pd.read_csv('carnet_dataset/train/metadata.csv')
train_metadata.head()

,car,bus,truck,train,motorcycle,bicycle,airplane,boat,traffic light,stop sign,vehicle,signal,file_name
0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,000000000064.jpg
1,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,000000000073.jpg
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,000000000074.jpg
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,000000000081.jpg
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,000000000086.jpg


## Score Images and Metadata

In [4]:
score_metadata = pd.read_csv('carnet_dataset/score/metadata.csv')
score_metadata.head()

,file_name
0,000000000071.jpg
1,000000000149.jpg
2,000000000260.jpg
3,000000000307.jpg
4,000000000690.jpg


## Creating Tensorflow Dataset (Single Image, Labeled Segment)

In [5]:
### Splitting the train data into train, validate, and test 
from sklearn.model_selection import train_test_split
train_metadata, test_metadata = train_test_split(train_metadata, test_size=0.20, random_state=42)
train_metadata, val_metadata = train_test_split(train_metadata, test_size=0.25, random_state=42)

In [6]:
# See Python Generator
# https://peps.python.org/pep-0255/
def build_generator_labeled_single(metadata: pd.DataFrame):
    def generator():
        for _, row in metadata.iterrows():
            
            training_path = 'carnet_dataset/train/images/' + row['file_name']
            training_np = np.array(Image.open(training_path)).astype(np.float32)
            
            ### Passing image as input
            model_input = training_np
            
            ### Passing count of vehicles and count of signals as outputs
            model_output_1 = (row["vehicle"], )
            model_output_2 = (row["signal"], )
            yield (model_input, (model_output_1, model_output_2))
            
    return generator


# See Tensorflow Dataset
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
def build_dataset_labeled_single(metadata: pd.DataFrame) -> tf.data.Dataset:
    model_input = tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32)  # type: ignore
    model_output = tf.TensorSpec(shape=(1, ), dtype=tf.float32)  # type: ignore

    dataset_signature = (model_input, (model_output, model_output))

    dataset = tf.data.Dataset.from_generator(
        build_generator_labeled_single(metadata), 
        output_signature=dataset_signature
    )

    return dataset


In [7]:
batch_size = 128

In [8]:
val_single_dataset = build_dataset_labeled_single(val_metadata).batch(batch_size)
# model_input, (model_output_1, model_output_2) = next(iter(val_single_dataset))
# display(model_input.shape, model_output_1.shape, model_output_2.shape)

2022-10-31 04:05:11.934278: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-31 04:05:11.935000: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 04:05:11.935925: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 04:05:11.936752: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so retur

In [9]:
test_single_dataset = build_dataset_labeled_single(test_metadata).batch(batch_size)
# model_input, (model_output_1, model_output_2) = next(iter(test_single_dataset))
# display(model_input.shape, model_output_1.shape, model_output_2.shape)

## Creating Tensorflow Dataset (Single Image, Score Segment)

In [10]:
import tensorflow as tf
import numpy as np

# See Python Generator
# https://peps.python.org/pep-0255/

def generator_score_single():
    metadata = pd.read_csv('carnet_dataset/score/metadata.csv')
    
    for _, row in metadata.iterrows():
        image_path = 'carnet_dataset/score/images/' + row['file_name']

        image_np = np.array(Image.open(image_path))

        model_input = image_np
        yield model_input


# See Tensorflow Dataset
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
def build_dataset_score_single() -> tf.data.Dataset:
    model_input = tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32)  # type: ignore

    dataset_signature = model_input

    dataset = tf.data.Dataset.from_generator(
        generator_score_single, 
        output_signature=dataset_signature
    )

    return dataset

In [11]:
score_single_dataset = build_dataset_score_single().batch(batch_size)
# model_input = next(iter(score_single_dataset))
# display(model_input.shape)

# Model

In [12]:
from tensorflow.keras.initializers import GlorotUniform

In [13]:
### Function to build individual models of the Ensemble
def build_model(kernel_seed):
    
    ### Getting pre-trained EfficientNetV2B1 Architecture, 
    ### using global average pooling at the end and excluding the top dense layer
    efficient_net = tf.keras.applications.EfficientNetV2B1(
        include_top=False,
        input_shape=(224, 224, 3),
        pooling="avg",
        weights="imagenet"
    )

    ### Freezing the parameters of the MobileNet layers
    efficient_net.trainable = False
    
    ### Input: image
    input = tf.keras.layers.Input((224, 224, 3))

    ### Using MobileNet for the image
    outputs = efficient_net(input)

    ### Creating two seperate models for Vehicles and Signals using Dense Layers
    outputs_1 = tf.keras.layers.Dense(64, activation='relu',
                                      kernel_initializer=GlorotUniform(seed=kernel_seed+100))(outputs)
    outputs_1 = tf.keras.layers.Dense(32, activation='relu',
                                      kernel_initializer=GlorotUniform(seed=kernel_seed+200))(outputs_1)
    outputs_1 = tf.keras.layers.Dense(1,
                                      kernel_initializer=GlorotUniform(seed=kernel_seed+300))(outputs_1)

    
    outputs_2 = tf.keras.layers.Dense(64, activation='relu',
                                      kernel_initializer=GlorotUniform(seed=kernel_seed+400))(outputs)
    outputs_2 = tf.keras.layers.Dense(32, activation='relu',
                                      kernel_initializer=GlorotUniform(seed=kernel_seed+500))(outputs_2)
    outputs_2 = tf.keras.layers.Dense(1,
                                      kernel_initializer=GlorotUniform(seed=kernel_seed+600))(outputs_2)

    model = tf.keras.models.Model(inputs=input, outputs=[outputs_1, outputs_2])

    ### Adam optimizer with 0.001 learning rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            lr=0.001
        ),
        ### MSE loss function
        loss='mean_squared_error'
    )
    return model

In [14]:
### Function to train individual models of the Ensemble
def train_model(model, train_single_dataset, val_single_dataset, seed):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    ### Checkpoint path 
    checkpoint_path = "weights_27/car_{0}".format(seed)
    checkpoint_dir = os.path.dirname(checkpoint_path)

    ### Callback to save model log / results after each epoch
    csv_logger = tf.keras.callbacks.CSVLogger("weights_27/model{0}_history_log.csv".format(seed), append=True)

    ### Callback to save weights after every epoch i.e., checkpoint
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                     save_weights_only=True,
                                                     verbose=1, save_best_only=True, monitor = 'val_loss') 

    ### Callback to stop the training if there's no improvement in validation loss for 3 epochs
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    
    ### Fitting the model
    model.fit(
        train_single_dataset,
        epochs=128,
        shuffle=True,
        validation_data=(val_single_dataset),
        callbacks=[stop_early, cp_callback, csv_logger]
    )
    
    return model

In [15]:
### Function to save individual models of the Ensemble
def save_model(model, seed):
    model_dir = 'weights_27/model{0}'.format(seed)
    os.makedirs(model_dir, exist_ok=True)
    with open(f'{model_dir}/keras_model.json', 'w') as f:
        model_json = json.dumps(json.loads(model.to_json()), indent=True)
        f.write(model_json)

    model.save_weights(f"{model_dir}/keras_parameters.h5")

In [16]:
### Function to predict and save individual models' outputs 
    ### for train, validate, test, and score in parquet files
def make_predictions(model, train, val, test, score, seed):
    model_dir = 'weights_27/model{0}'.format(seed)
    
    y_hat = model.predict(train)
    y_hat = pd.DataFrame(np.array(y_hat).reshape(2, -1).transpose(), columns = ["vehicle", "signal"])
    y_hat.to_parquet(f'{model_dir}/train_y_hat.parquet')
    
    y_hat = model.predict(val)
    y_hat = pd.DataFrame(np.array(y_hat).reshape(2, -1).transpose(), columns = ["vehicle", "signal"])
    y_hat.to_parquet(f'{model_dir}/val_y_hat.parquet')
    
    y_hat = model.predict(test)
    y_hat = pd.DataFrame(np.array(y_hat).reshape(2, -1).transpose(), columns = ["vehicle", "signal"])
    y_hat.to_parquet(f'{model_dir}/test_y_hat.parquet')
    
    y_hat = model.predict(score)
    y_hat = pd.DataFrame(np.array(y_hat).reshape(2, -1).transpose(), columns = ["vehicle", "signal"])
    y_hat.to_parquet(f'{model_dir}/score_y_hat.parquet')

In [17]:
### Creating 7 models
model_dir = 'weights_27'
os.makedirs(model_dir, exist_ok=True)
np.random.seed(123)
n_models = 7
seeds = np.random.randint(0, 1e2, size=n_models)

In [ ]:
### Train individual models
for seed in seeds:
    print(seed)
    ### Random resampling of the dataset with replacement (bootstrap replicate)
    train_data = train_metadata.sample(frac = 1, replace = True, random_state = seed)
    train_single_dataset = build_dataset_labeled_single(train_data).batch(batch_size)
    model = build_model(seed)
    model = train_model(model, train_single_dataset, val_single_dataset, seed)
    save_model(model, seed)

66
Epoch 1/128


/home/ec2-user/SageMaker/setup/miniconda/envs/ucla_deeplearning/lib/python3.10/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2022-10-31 00:03:45.922000: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8401
2022-10-31 00:03:46.454267: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.0.145, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


    109/Unknown - 86s 687ms/step - loss: 11.9509 - dense_3_loss: 10.4316 - dense_6_loss: 1.5193
Epoch 1: val_loss improved from inf to 9.66044, saving model to weights_27/car_66
109/109 [==============================] - 114s 947ms/step - loss: 11.9509 - dense_3_loss: 10.4316 - dense_6_loss: 1.5193 - val_loss: 9.6604 - val_dense_3_loss: 8.3635 - val_dense_6_loss: 1.2969
Epoch 2/128
109/109 [==============================] - ETA: 0s - loss: 8.6230 - dense_3_loss: 7.5001 - dense_6_loss: 1.1229
Epoch 2: val_loss improved from 9.66044 to 9.35475, saving model to weights_27/car_66
109/109 [==============================] - 98s 905ms/step - loss: 8.6230 - dense_3_loss: 7.5001 - dense_6_loss: 1.1229 - val_loss: 9.3547 - val_dense_3_loss: 8.0747 - val_dense_6_loss: 1.2800
Epoch 3/128
109/109 [==============================] - ETA: 0s - loss: 6.9881 - dense_3_loss: 6.1378 - dense_6_loss: 0.8503
Epoch 3: val_loss did not improve from 9.35475
109/109 [==============================] - 97s 894ms/s

In [28]:
### Making predictions on the whole train set and validation, test, and score sets
train_single_dataset = build_dataset_labeled_single(train_metadata).batch(batch_size)
for seed in seeds:
    model_dir = 'weights_27/model{0}'.format(seed)
    model = json.load(open(f'{model_dir}/keras_model.json'))
    model = tf.keras.models.model_from_config(model)
    model.load_weights(f"{model_dir}/keras_parameters.h5")
    make_predictions(model, train_single_dataset, val_single_dataset, test_single_dataset, 
                     score_single_dataset, seed)

In [18]:
### Getting predictions of all models and adding them to the list as inputs to the ensemble model
train_pred = []
val_pred = []
test_pred = []
score_pred = []
for seed in seeds:
    model_dir = 'weights_27/model{0}/'.format(seed)
    train_pred.append(pd.read_parquet(f'{model_dir}/train_y_hat.parquet', engine='pyarrow').values)
    val_pred.append(pd.read_parquet(f'{model_dir}/val_y_hat.parquet', engine='pyarrow').values)
    test_pred.append(pd.read_parquet(f'{model_dir}/test_y_hat.parquet', engine='pyarrow').values)
    score_pred.append(pd.read_parquet(f'{model_dir}/score_y_hat.parquet', engine='pyarrow').values)

### Train
ip1 = np.array(train_pred).transpose()[0,:,:]
ip2 = np.array(train_pred).transpose()[1,:,:]
op1 = train_metadata["vehicle"].values
op2 = train_metadata["signal"].values

### Validate
ip1v = np.array(val_pred).transpose()[0,:,:]
ip2v = np.array(val_pred).transpose()[1,:,:]
op1v = val_metadata["vehicle"].values
op2v = val_metadata["signal"].values

### Test
ip1t = np.array(test_pred).transpose()[0,:,:]
ip2t = np.array(test_pred).transpose()[1,:,:]
op1t = test_metadata["vehicle"].values
op2t = test_metadata["signal"].values

### Score
ip1s = np.array(score_pred).transpose()[0,:,:]
ip2s = np.array(score_pred).transpose()[1,:,:]

In [87]:
### Creating the Ensemble model with outputs of 7 inidividual models as inputs
model_input_1 = tf.keras.layers.Input((7))  ### Vehicles
model_input_2 = tf.keras.layers.Input((7))  ### Signals

### Creating individual models for vehicles and signals with dense layers and dropouts to reduce overfitting
ensemble_output_1 = tf.keras.layers.Dense(128, activation = "relu")(model_input_1)
ensemble_output_1 = tf.keras.layers.Dropout(0.25)(ensemble_output_1)
ensemble_output_1 = tf.keras.layers.Dense(1)(ensemble_output_1)

ensemble_output_2 = tf.keras.layers.Dense(128, activation = "relu")(model_input_2)
ensemble_output_2 = tf.keras.layers.Dropout(0.25)(ensemble_output_2)
ensemble_output_2 = tf.keras.layers.Dense(1)(ensemble_output_2)

ensemble_model = tf.keras.Model(inputs=[model_input_1, model_input_2], 
                                outputs=[ensemble_output_1, ensemble_output_2])

In [88]:
### Checkpoint path 
checkpoint_path = "weights/car_27"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [89]:
### Adam optimizer with 0.001 learning rate
ensemble_model.compile(
    optimizer=tf.keras.optimizers.Adam(
        lr=0.001
    ),
    ### MSE loss
    loss='mean_squared_error'
)

In [82]:
### Training the Ensemble model
np.random.seed(11)
tf.random.set_seed(11)

### Callback to save model log / results after each epoch
csv_logger = tf.keras.callbacks.CSVLogger("model27_history_log.csv", append=True)

### Callback to save weights after every epoch i.e., checkpoint
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1, save_best_only=True, monitor = 'val_loss') 

### Callback to stop the training if there's no improvement in validation loss for 3 epochs
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
ensemble_model.fit(
    x = [ip1, ip2],
    y = [op1, op2],
    epochs=128,
    shuffle=True,
    validation_data=([ip1v, ip2v], [op1v, op2v]),
    callbacks=[stop_early, cp_callback, csv_logger]
)

Epoch 1/128
427/436 [============================>.] - ETA: 0s - loss: 5.5672 - dense_45_loss: 4.9973 - dense_47_loss: 0.5699
Epoch 1: val_loss improved from inf to 8.60365, saving model to weights/car_271
436/436 [==============================] - 3s 5ms/step - loss: 5.5533 - dense_45_loss: 4.9848 - dense_47_loss: 0.5685 - val_loss: 8.6037 - val_dense_45_loss: 7.4257 - val_dense_47_loss: 1.1779
Epoch 2/128
426/436 [============================>.] - ETA: 0s - loss: 4.9869 - dense_45_loss: 4.4855 - dense_47_loss: 0.5013
Epoch 2: val_loss did not improve from 8.60365
436/436 [==============================] - 2s 4ms/step - loss: 4.9794 - dense_45_loss: 4.4773 - dense_47_loss: 0.5021 - val_loss: 8.7114 - val_dense_45_loss: 7.5336 - val_dense_47_loss: 1.1779
Epoch 3/128
432/436 [============================>.] - ETA: 0s - loss: 4.7844 - dense_45_loss: 4.3029 - dense_47_loss: 0.4815
Epoch 3: val_loss did not improve from 8.60365
436/436 [==============================] - 2s 4ms/step - loss:

In [90]:
### Loading the weights from the best model
ensemble_model.load_weights(checkpoint_path)

In [91]:
### Evaluating model on Validation
total_loss, _, _ = ensemble_model.evaluate([ip1v, ip2v], [op1v, op2v])
total_loss/2

146/146 [==============================] - 1s 2ms/step - loss: 8.5805 - dense_49_loss: 7.4031 - dense_51_loss: 1.1774


4.29025936126709

In [92]:
### Evaluating model on Test
total_loss, _, _ = ensemble_model.evaluate([ip1t, ip2t], [op1t, op2t])
total_loss/2

146/146 [==============================] - 0s 2ms/step - loss: 8.7264 - dense_49_loss: 7.4870 - dense_51_loss: 1.2394


4.363196849822998

In [94]:
### Predicting on Score
score = ensemble_model.predict([ip1s, ip2s])

In [95]:
score_y_hat = pd.DataFrame(np.array(score).reshape(2, -1).transpose(), columns = ["vehicle", "signal"])
score_y_hat = score_y_hat[["signal", "vehicle"]]

In [96]:
# Use the following asserts to check the type and shape of the final predictions.
assert type(score_y_hat) == pd.DataFrame
assert score_y_hat.shape == (score_metadata.shape[0], 2)
assert (score_y_hat.columns == ['signal', 'vehicle']).all()

In [83]:
# Use the following code to save the final predictions.
import os 
model_dir = 'carnet_model'
os.makedirs(model_dir, exist_ok=True)
score_y_hat.to_parquet(f'{model_dir}/score_y_hat.parquet')

In [93]:
### Saving the Ensemble model
with open(f'{model_dir}/keras_model.json', 'w') as f:
    ensemble_model_json = json.dumps(json.loads(ensemble_model.to_json()), indent=True)
    f.write(ensemble_model_json)
ensemble_model.save_weights(f"{model_dir}/keras_parameters.h5")

#### Alternate Way for Ensemble (Takes More Time)

In [97]:
### Load all individual models
models = []
for seed in seeds:
    model_dir = 'weights_27/model{0}'.format(seed)
    model = json.load(open(f'{model_dir}/keras_model.json'))
    model = tf.keras.models.model_from_config(model)
    model.load_weights(f"{model_dir}/keras_parameters.h5")
    ### Freeze the layers
    model.trainable = False
    models.append(model)

In [98]:
### Image as the input
model_input = tf.keras.layers.Input((224, 224, 3))

### Outputs of individual models
model_outputs_1 = [model(model_input)[0] for model in models] ### Vehicles
model_outputs_2 = [model(model_input)[1] for model in models] ### Signals

### Creating individual models for vehicles and signals with dense layers and dropouts to reduce overfitting
ensemble_output_1 = tf.keras.layers.Concatenate()(model_outputs_1)
ensemble_output_1 = tf.keras.layers.Dense(128, activation = "relu")(ensemble_output_1)
ensemble_output_1 = tf.keras.layers.Dropout(0.25)(ensemble_output_1)
ensemble_output_1 = tf.keras.layers.Dense(1)(ensemble_output_1)

ensemble_output_2 = tf.keras.layers.Concatenate()(model_outputs_2)
ensemble_output_2 = tf.keras.layers.Dense(128, activation = "relu")(ensemble_output_2)
ensemble_output_2 = tf.keras.layers.Dropout(0.25)(ensemble_output_2)
ensemble_output_2 = tf.keras.layers.Dense(1)(ensemble_output_2)

ensemble_model = tf.keras.Model(inputs=model_input, outputs=[ensemble_output_1, ensemble_output_2])

In [25]:
ensemble_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 model (Functional)             [(None, 1),          7099318     ['input_1[0][0]',                
                                 (None, 1)]                       'input_1[0][0]']                
                                                                                                  
 model_1 (Functional)           [(None, 1),          7099318     ['input_1[0][0]',                
                                 (None, 1)]                       'input_1[0][0]']            

In [99]:
### Checkpoint path 
checkpoint_path = "weights/car_27"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [100]:
### Adam optimizer with 0.001 learning rate
ensemble_model.compile(
    optimizer=tf.keras.optimizers.Adam(
        lr=0.001
    ),
    ### MSE loss
    loss='mean_squared_error'
)

/home/ec2-user/SageMaker/setup/miniconda/envs/ucla_deeplearning/lib/python3.10/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [21]:
### Whole train data
train_single_dataset = build_dataset_labeled_single(train_metadata).batch(batch_size)

In [ ]:
### Training the Ensemble model
np.random.seed(11)
tf.random.set_seed(11)

### Callback to save model log / results after each epoch
csv_logger = tf.keras.callbacks.CSVLogger("model27_2_history_log.csv", append=True)

### Callback to save weights after every epoch i.e., checkpoint
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1, save_best_only=True, monitor = 'val_loss') 

### Callback to stop the training if there's no improvement in validation loss for 3 epochs
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

### Fitting the model
ensemble_model.fit(
    train_single_dataset,
    epochs=128,
    shuffle=True,
    validation_data=(val_single_dataset),
    callbacks=[stop_early, cp_callback, csv_logger]
)

Epoch 1/128


2022-10-31 02:44:50.732480: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8401
2022-10-31 02:44:51.353386: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.0.145, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


    109/Unknown - 794s 6s/step - loss: 7.2389 - dense_2_loss: 6.4384 - dense_3_loss: 0.8005
Epoch 1: val_loss improved from inf to 8.73772, saving model to weights/car_27_2
109/109 [==============================] - 1061s 9s/step - loss: 7.2389 - dense_2_loss: 6.4384 - dense_3_loss: 0.8005 - val_loss: 8.7377 - val_dense_2_loss: 7.4917 - val_dense_3_loss: 1.2460
Epoch 2/128
109/109 [==============================] - ETA: 0s - loss: 5.4629 - dense_2_loss: 4.9267 - dense_3_loss: 0.5361
Epoch 2: val_loss did not improve from 8.73772
109/109 [==============================] - 919s 8s/step - loss: 5.4629 - dense_2_loss: 4.9267 - dense_3_loss: 0.5361 - val_loss: 8.8938 - val_dense_2_loss: 7.6510 - val_dense_3_loss: 1.2428
Epoch 3/128
 15/109 [===>..........................] - ETA: 9:57 - loss: 5.0414 - dense_2_loss: 4.5256 - dense_3_loss: 0.5158 

KeyboardInterrupt: 

In [101]:
### Loading the weights from the best model
ensemble_model.load_weights(checkpoint_path)

In [29]:
### Evaluating model on Validation
ensemble_model.evaluate(val_single_dataset)

37/37 [==============================] - 230s 6s/step - loss: 8.7377 - dense_2_loss: 7.4917 - dense_3_loss: 1.2460


[8.737717628479004, 7.491723537445068, 1.2459948062896729]

In [30]:
### Evaluating model on Test
ensemble_model.evaluate(test_single_dataset)

37/37 [==============================] - 230s 6s/step - loss: 8.8654 - dense_2_loss: 7.5516 - dense_3_loss: 1.3138


[8.8654146194458, 7.551567554473877, 1.3138481378555298]

In [32]:
score = ensemble_model.predict(score_single_dataset)

In [38]:
score_y_hat = pd.DataFrame(np.array(score).reshape(2, -1).transpose(), columns = ["vehicle", "signal"])
score_y_hat = score_y_hat[["signal", "vehicle"]]

In [83]:
# Use the following code to save the final predictions.
import os 
model_dir = 'carnet_model'
os.makedirs(model_dir, exist_ok=True)
score_y_hat.to_parquet(f'{model_dir}/score_y_hat.parquet')

In [102]:
### Saving the Ensemble model
with open(f'{model_dir}/keras_model.json', 'w') as f:
    ensemble_model_json = json.dumps(json.loads(ensemble_model.to_json()), indent=True)
    f.write(ensemble_model_json)
ensemble_model.save_weights(f"{model_dir}/keras_parameters.h5")